In [1]:
import pandas as pd
df = pd.read_csv('train_data.csv')
df

C:\Users\Fatemeh\AppData\Local\Temp\ipykernel_29272\2546589644.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train_data.csv')


,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,2,NaN,False,2016-11-11,A2OSUEZJIN7BI,0511189877,NaN,Chris,I have an older URC-WR7 remote and thought thi...,Cannot Learn,1478822400
1,5,NaN,True,2016-06-06,A2NETQRG6JHIG7,0511189877,NaN,Qrysta White,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!?,1465171200
2,4,NaN,True,2016-03-10,A12JHGROAX49G7,0511189877,NaN,Linwood,Got them and only 2 of them worked. company ca...,Works Good and programs easy.,1457568000
3,5,NaN,True,2016-01-14,A1KV65E2TMMG6F,0511189877,NaN,Dane Williams,I got tired of the remote being on the wrong s...,Same as TWC remote,1452729600
4,5,NaN,True,2016-10-20,A280POPEWI0NSA,0594459451,NaN,Kristina H.,After purchasing cheap cords from another webs...,Good Quality Cord,1476921600
...,...,...,...,...,...,...,...,...,...,...,...
838939,5,NaN,True,2017-07-25,A1OOVLE2KZ6KGA,B01HJCN1EI,NaN,Puddzee,These are my favorite charging cords for a few...,Worth the price.,1500940800
838940,1,NaN,True,2017-04-04,A77K1B31UAQ29,B01HJCN1EI,NaN,addictedtoreading,"Update....after 2 months of gentle use, cable ...",UPDATE...BREAKS AND SLOW CHARGING,1491264000
838941,3,NaN,True,2017-07-08,A2SVXUVUAWUDK2,B01HJH42KU,NaN,Andrew,These are okay. The connection becomes very if...,Hope this makes sense. You'd understand if you...,1499472000
838942,5,2.0,True,2016-12-01,A2HUZO7MQAY5I2,B01HJH40WU,NaN,michael clontz,"Ok here is an odd thing that happened to me, I...",Not the correct product as linked in the sale.,1480550400


In [3]:
import re
df['cleaned_text'] = df['reviewText'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x).lower()))


In [4]:
from gensim.models import Word2Vec

tokenized_reviews = [text.split() for text in df['cleaned_text']]

params = [
    {'vector_size': 100, 'window': 5, 'min_count': 5, 'workers': 4},
    {'vector_size': 150, 'window': 4, 'min_count': 3, 'workers': 2},
    {'vector_size': 200, 'window': 10, 'min_count': 2, 'workers': 4},
    {'vector_size': 200, 'window': 20, 'min_count': 2, 'workers': 4},
    {'vector_size': 300, 'window': 10, 'min_count': 1, 'workers': 4}
]

models = []
for param in params:
    model = Word2Vec(sentences=tokenized_reviews, vector_size=param['vector_size'], window=param['window'], 
                     min_count=param['min_count'], workers=param['workers'])
    models.append(model)


In [25]:
all_similar_words = []

for model in models:

    warranty_similar_words = [word for word, _ in model.wv.most_similar('warranty', topn=10)]
    guarantee_similar_words = [word for word, _ in model.wv.most_similar('guarantee', topn=10)]
    all_similar_words.extend(warranty_similar_words + guarantee_similar_words)

keywords = list(set(all_similar_words + ['warranty', 'guarantee']))
keywords


['warrantied', 'contract', 'rebate', 'promise', 'assurant', 'guarantees', 'guarentee', 'guarantee', 'policy', 'claim', 'nextbusinessday', 'waranty', 'squaretrade', 'warranties', 'deny', 'warrantee', 'offer', 'guaranteed', 'assure', 'warantee', 'honor', 'warrantya', 'warranty', 'commitments', 'replacementsa', 'guaranty', 'warrenty']


In [26]:
keywords.remove('contract')
keywords.remove('offer')
keywords.remove('nextbusinessday')
keywords.remove('rebate')
keywords

['warrantied',
 'promise',
 'assurant',
 'guarantees',
 'guarentee',
 'guarantee',
 'policy',
 'claim',
 'waranty',
 'squaretrade',
 'warranties',
 'deny',
 'warrantee',
 'guaranteed',
 'assure',
 'warantee',
 'honor',
 'warrantya',
 'warranty',
 'commitments',
 'replacementsa',
 'guaranty',
 'warrenty']

In [27]:
len(models)
new_keywords = keywords.copy()
new_keywords

['warrantied',
 'promise',
 'assurant',
 'guarantees',
 'guarentee',
 'guarantee',
 'policy',
 'claim',
 'waranty',
 'squaretrade',
 'warranties',
 'deny',
 'warrantee',
 'guaranteed',
 'assure',
 'warantee',
 'honor',
 'warrantya',
 'warranty',
 'commitments',
 'replacementsa',
 'guaranty',
 'warrenty']

In [30]:
for model in models:
    print('....................')
    for word in keywords:
        try:
            print(word)
            similar_words = [word for word, similarity in model.wv.most_similar(word, topn=5)]
            new_keywords.extend(similar_words)  
        except KeyError:
            continue

new_keywords = list(set(new_keywords))
new_keywords  


....................
warrantied
promise
assurant
guarantees
guarentee
guarantee
policy
claim
waranty
squaretrade
warranties
deny
warrantee
guaranteed
assure
warantee
honor
warrantya
warranty
commitments
replacementsa
guaranty
warrenty
....................
warrantied
promise
assurant
guarantees
guarentee
guarantee
policy
claim
waranty
squaretrade
warranties
deny
warrantee
guaranteed
assure
warantee
honor
warrantya
warranty
commitments
replacementsa
guaranty
warrenty
....................
warrantied
promise
assurant
guarantees
guarentee
guarantee
policy
claim
waranty
squaretrade
warranties
deny
warrantee
guaranteed
assure
warantee
honor
warrantya
warranty
commitments
replacementsa
guaranty
warrenty
....................
warrantied
promise
assurant
guarantees
guarentee
guarantee
policy
claim
waranty
squaretrade
warranties
deny
warrantee
guaranteed
assure
warantee
honor
warrantya
warranty
commitments
replacementsa
guaranty
warrenty
....................
warrantied
promise
assurant
guarantees


['liable',
 'rebate',
 'cr2430',
 'runarounds',
 'sells',
 'authorize',
 'claiming',
 'discontinue',
 'warrantybut',
 'warrantythats',
 'sugarcoat',
 'honour',
 'assured',
 'policy',
 'agent',
 'claim',
 'fees',
 'honoring',
 'loveblue',
 'automount',
 'refundreplacement',
 'comeif',
 'claims',
 'warenty',
 '08092017',
 'reimbursement',
 'regulatory',
 'expire',
 'warrantya',
 'warranty',
 'commitments',
 'warn',
 'guaranty',
 'disrespect',
 'frill',
 '30day',
 'junsi',
 'warrantied',
 'promise',
 'certification',
 'hildatempered',
 'unwilling',
 'assurant',
 'doomed',
 'guarentee',
 'approve',
 'zilch',
 'blackouta',
 'guarantee',
 'bs',
 'deadline',
 'applecare',
 'refunds',
 'plugunplugging',
 'tell',
 'warranties',
 'oneyear',
 'scam',
 'willing',
 'warrantee',
 'denying',
 'predict',
 'responsible',
 '599',
 'brainier',
 'honored',
 'attest',
 'hrefmicrosoftwirelessmobilemouse3500yearofthedragongolddpb005x64o7qrefcm_cr_arp_d_rvw_txtieutf8microsoft',
 'asurion',
 'inform',
 'adh',


In [31]:
len(new_keywords)

133

In [33]:
new_keywords

{'08092017',
 '10anbspto',
 '30day',
 '599',
 '7000tm',
 'acknowledge',
 'adh',
 'advertise',
 'affiliations',
 'agent',
 'amazons',
 'antibubble',
 'applecare',
 'approve',
 'argue',
 'argued',
 'assurant',
 'assure',
 'assured',
 'asurion',
 'attest',
 'authorization',
 'authorize',
 'automount',
 'believe',
 'blackouta',
 'bound',
 'brainier',
 'bs',
 'certification',
 'certified',
 'claim',
 'claimed',
 'claiming',
 'claims',
 'comeif',
 'commitment',
 'commitments',
 'confirm',
 'contract',
 'cr2430',
 'dayvue',
 'deadline',
 'deny',
 'denying',
 'destined',
 'discontinue',
 'disrespect',
 'doomed',
 'ensures',
 'exhibitor',
 'expire',
 'fathom',
 'fees',
 'forgive',
 'frill',
 'guarantee',
 'guaranteed',
 'guarantees',
 'guaranty',
 'guarentee',
 'hickups',
 'hildatempered',
 'honor',
 'honored',
 'honoring',
 'honour',
 'hope',
 'hreflogitechm325wirelessopticalmousewithdesignedforwebscrollingfreespirit910004160dpb00kk928lurefcm_cr_arp_d_rvw_txtieutf8logitech',
 'hrefmicrosoftwir

In [35]:
new_keywords = {
 'assurant',
 'assure',
 'assured',
 'asurion',
 'claim',
 'claimed',
 'claiming',
 'claims',
 'commitment',
 'commitments',
 'contract',
 'deny',
 'denying',
 'ensure',
 'expire',
 'guarantee',
 'guaranteed',
 'guarantees',
 'guaranty',
 'guarentee',
 'honor',
 'honored',
 'honoring',
 'honour',
 'nextbusinessday',
 'policy',
 'policies',
 'promise',
 'promises',
 'rebate',
 'refundreplacement',
 'replacementsa',
 'replacementa',
 'squaretrade',
 'warantee',
 'waranty',
 'warenty',
 'warrantech',
 'warrantee',
 'warrantied',
 'warranties',
 'warranty',
 'warrantybut',
 'warrenty'
}

new_keywords.remove('rebate')
new_keywords.remove('nextbusinessday')
new_keywords.remove('asurion')
new_keywords.remove('denying')
new_keywords.remove('expire')
new_keywords.remove('honoring')
new_keywords.remove('squaretrade')
new_keywords.remove('claiming')
new_keywords.remove('contract')
new_keywords.remove('honour')
new_keywords.remove('promises')
new_keywords, len(new_keywords)

({'assurant',
  'assure',
  'assured',
  'claim',
  'claimed',
  'claims',
  'commitment',
  'commitments',
  'deny',
  'ensure',
  'guarantee',
  'guaranteed',
  'guarantees',
  'guaranty',
  'guarentee',
  'honor',
  'honored',
  'policies',
  'policy',
  'promise',
  'refundreplacement',
  'replacementa',
  'replacementsa',
  'warantee',
  'waranty',
  'warenty',
  'warrantech',
  'warrantee',
  'warrantied',
  'warranties',
  'warranty',
  'warrantybut',
  'warrenty'},
 33)

In [40]:
def contains_keyword(review, keywords):
    for word in keywords:
        if word in review.split():
            return True
    return False

df['contains_warranty'] = df['cleaned_text'].apply(lambda x: contains_keyword(x, new_keywords))

warranty_reviews = df[df['contains_warranty'] == True]
warranty_reviews[['reviewText', 'overall']]


,reviewText,overall
28,"I originally bought this for a Sony 40"" TV, bu...",5
55,I was going to deploy to my third deployment t...,5
78,For the most part this was a very sweet story....,3
155,My Wife & I are in the stage of life where we ...,5
167,This adapter works well with my 1994 Corvette....,4
...,...,...
838879,This is a very attractive and solidly made cab...,1
838886,"5 stars, ""Fast, reliable SSD, even externally....",5
838891,The XTi series is Corsair's flagship SSD. The ...,5
838892,This inexpensive reader worked on my Galaxy S7...,5


In [44]:
avg_ratings_df = warranty_reviews.groupby('asin')['overall'].mean().reset_index()
avg_ratings_df.to_csv('average_ratings.csv', index=False)
avg_ratings_df

,asin,overall
0,0972683275,5.0
1,140053271X,5.0
2,1495443043,3.0
3,6541654530,1.0
4,7564005068,5.0
...,...,...
16437,B01HIS5O7A,2.0
16438,B01HIURQWE,4.5
16439,B01HIWBU7Y,5.0
16440,B01HIZEW1C,5.0
